In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np

def elimination_matrix(A: np.ndarray, step: int):
    
    n = A.shape[0]
    elim_mtx = np.eye(n)
    elim_mtx_inv = np.eye(n)

    if 0 < step < n:
        a = A[:, step-1] / A[step-1, step-1]
        elim_mtx[step:, step-1] = -a[step:]
        elim_mtx_inv[step:, step-1] = a[step:]
    
    return elim_mtx, elim_mtx_inv

In [3]:
def LU(A: np.ndarray):

    n = A.shape[0]
    L = np.eye(n)
    U = np.copy(A)

    for step in range(1, n):
        elim_mtx, elim_mtx_inv = elimination_matrix(U, step=step)
        U = np.matmul(elim_mtx, U)
        L = np.matmul(L, elim_mtx_inv)

    return L, U

In [4]:
A = 10 * np.random.rand(4, 4) - 5
A

array([[ 0.63255196,  4.85279357, -0.31936795,  2.56680738],
       [-3.61963577,  1.14684117,  2.82953888,  4.84590546],
       [-1.19068101, -0.86831605, -4.17400032, -3.66430562],
       [-3.89863578,  3.53805829, -4.49538968, -1.00887098]])

In [5]:
L, U = LU(A)

print(f"Lower: \n{L}\n\nUpper:\n{U}")

Lower: 
[[ 1.          0.          0.          0.        ]
 [-5.72227424  1.          0.          0.        ]
 [-1.882345    0.28587483  1.          0.        ]
 [-6.16334474  1.15671821  1.50600699  1.        ]]

Upper:
[[ 6.32551957e-01  4.85279357e+00 -3.19367950e-01  2.56680738e+00]
 [ 0.00000000e+00  2.89158568e+01  1.00202789e+00  1.95338812e+01]
 [ 0.00000000e+00  0.00000000e+00 -5.06161553e+00 -4.41693356e+00]
 [ 0.00000000e+00  0.00000000e+00 -8.88178420e-16 -1.13201545e+00]]


In [6]:
np.allclose(np.matmul(L, U), A)

True

In [11]:
def invert_lower_triangular_matrix(L: np.ndarray):

    n = L.shape[0]
    G = np.eye(n)
    D = np.copy(L)

    for step in range(1, n):
        elim_mtx, _ = elimination_matrix(D, step=step)
        G = np.matmul(elim_mtx, G)
        D = np.matmul(elim_mtx, D)

    D_inv = np.eye(n) / np.diagonal(D)

    return np.matmul(D_inv, G)

In [12]:
def invert(A: np.ndarray):
    
    L, U = LU(A)
    L_inv = invert_lower_triangular_matrix(L)
    U_inv = invert_lower_triangular_matrix(U.T).T

    return np.matmul(U_inv, L_inv)


In [13]:
A = np.random.rand(3, 3)
A_inv = invert(A)

print(f"A:\n{A}\n\nA⁻¹:\n{A_inv}\n\nAA⁻¹:\n{np.matmul(A, A_inv)}")

A:
[[0.99598321 0.9018686  0.09594627]
 [0.9807353  0.2664219  0.26306197]
 [0.74674113 0.80602621 0.1934162 ]]

A⁻¹:
[[ 1.65441318  1.00087412 -2.18196142]
 [-0.06956683 -1.24713736  1.73071897]
 [-6.09745031  1.33303997  6.38186194]]

AA⁻¹:
[[ 1.00000000e+00  9.38769659e-17 -5.82419239e-17]
 [-2.33374635e-16  1.00000000e+00  3.29754113e-16]
 [-1.86560999e-17  2.36264448e-17  1.00000000e+00]]


In [14]:
for _ in range(1000):
    n = np.random.randint(1, 10)
    A = np.random.rand(n, n)
    A_inv = invert(A)
    if not np.allclose(np.matmul(A, A_inv), np.eye(n), atol=1e-5):
        print("Test failed.")

In [15]:

A = np.random.rand(3, 3)
A_inv = np.linalg.inv(A)

print(f"A:\n{A}\n\nNumPy's A⁻¹:\n{A_inv}\n\nAA⁻¹:\n{np.matmul(A, A_inv)}")

A:
[[0.60906127 0.46898225 0.60823081]
 [0.66858205 0.02672382 0.70405557]
 [0.76564027 0.80417909 0.63000603]]

NumPy's A⁻¹:
[[-15.58256266   5.49338566   8.90490831]
 [  3.34265451  -2.32521949  -0.62859989]
 [ 14.67056529  -3.70800319  -8.43237882]]

AA⁻¹:
[[ 1.00000000e+00 -1.68795671e-16  3.54932360e-16]
 [ 5.66618363e-16  1.00000000e+00  1.15919087e-15]
 [ 8.78580914e-16  6.66039086e-17  1.00000000e+00]]


In [16]:
from timeit import timeit


n_runs = 100
size = 100
A = np.random.rand(size, size)

t_inv = timeit(lambda: invert(A), number=n_runs)
t_np_inv = timeit(lambda: np.linalg.inv(A), number=n_runs)


print(f"Our invert:              \t{t_inv} s")
print(f"NumPy's invert:          \t{t_np_inv} s")
print(f"Performance improvement: \t{t_inv/t_np_inv} times faster")

Our invert:              	4.54282830003649 s
NumPy's invert:          	0.4907903999555856 s
Performance improvement: 	9.256147431668582 times faster


In [20]:
def det(A: np.ndarray):

    n, m = A.shape

    if n != m:
        raise ValueError("A must be a square matrix")
    
    if n == 1:
        return A[0, 0]
    else:
        return sum([(-1)**j*A[0, j] * det(np.delete(A[1:], j, axis=1)) for j in range(n)])  

In [21]:
A = np.array([[1, 2],
              [3, 4]
])

In [22]:
det(A)

np.int64(-2)

In [23]:
from timeit import timeit

A = np.random.rand(10, 10)
t_det = timeit(lambda: det(A), number=1)

print(f"The time it takes to compute the determinant of a 10 x 10 matrix: {t_det} seconds")

The time it takes to compute the determinant of a 10 x 10 matrix: 49.92864189995453 seconds


In [24]:
def fast_det(A: np.ndarray):
    L, U = LU(A)
    return np.prod(np.diag(U))

In [25]:
A = np.random.rand(10, 10)


t_fast_det = timeit(lambda : fast_det(A), number=1)
print(f"The time it takes to compute the determinant of a 10 x 10 matrix: {t_fast_det} seconds")

The time it takes to compute the determinant of a 10 x 10 matrix: 0.00020820018835365772 seconds


In [26]:
print(f"Recursive determinant:   \t{t_det} s")
print(f"LU determinant:          \t{t_fast_det} s")
print(f"Performance improvement: \t{t_det/t_fast_det} times faster")

Recursive determinant:   	49.92864189995453 s
LU determinant:          	0.00020820018835365772 s
Performance improvement: 	239810.74318311186 times faster


In [27]:
from copy import deepcopy


def permutations(n: int):
    if n == 0:
        return [[0]]
    else:
        prev_permutations = permutations(n - 1)
        
        new_permutations = []
        
        for p in prev_permutations:
            for i in range(len(p)+1):
                p_new = deepcopy(p)
                p_new.insert(i, n)
                new_permutations.append(p_new)
                
        return new_permutations

In [28]:
from itertools import product


def inversion(permutation: list):
    n = len(permutation)
    inversions = sum([1 for i, j in product(range(n), range(n)) if i < j and permutation[i] > permutation[j]])
    return inversions

In [29]:
def sign(permutation: list):
    i = inversion(permutation)
    return (-1)**i

In [30]:
def permutation_formula(A: np.ndarray):
    n, _ = A.shape
    S_n = permutations(n-1)
    determinant = sum([sign(p)*np.prod([A[p[i], i] for i in range(n)]) for p in S_n])
    return determinant